In [1]:
import pandas as pd
import numpy as np
import pydeck as pdk

In [2]:
# Lightstation data
ls_data = pd.read_csv('./data/BC_Lightstations_and_other_Sample_Sites.csv', skiprows=1)
ls_data.rename(columns={
    'LATITUDE (DECIMAL DEGREES)': 'latitude',
    'Unnamed: 4': 'longitude',
    'LIGHTSTATION / LOCATION': 'lightstation'
}, inplace=True)
ls_data.rename(str.lower, axis='columns', inplace=True)
ls_active = ls_data[ls_data['data collection status']=='ACTIVE']
ls_inactive = ls_data[ls_data['data collection status']!='ACTIVE']


In [31]:
midpoint = (np.median(ff_sites["latitude"]), np.median(ff_sites["longitude"]))

map = pdk.Deck(
    initial_view_state={
        "latitude": midpoint[0],
        "longitude": midpoint[1],
        "zoom": 6,
        "pitch": 50,
    },
    layers=[
        pdk.Layer(
        "ScatterplotLayer",
        ls_active,
        get_position=['longitude', 'latitude'],
        auto_highlight=True,
        get_radius=2000,
        pickable=True,
        get_fill_color=[255, 0, 0, 255],
        ),
    ],
    mapbox_key='pk.eyJ1IjoidG9tc2l0dGVyIiwiYSI6ImNrYjJocjU0azBiNTEycnFwZWFkajB2bnoifQ.M1BDSPntGHfosotYAUAINw'
)

map.to_html('farms.html')

'C:\\Users\\Tom\\Projects\\sea-lice\\farms.html'

In [3]:
# Fish farm data
ff_data = pd.read_csv('./data/lice-count-dens-pou-2011-ongoing-rpt-pac-dfo-mpo-aquaculture-eng.csv')
ff_data.columns
ff_data.rename(columns = {
    'Latitude (Decimal Degrees)': 'latitude',
    'Longitude (Decimal Degrees)': 'longitude',
    'Facility Reference \nNumber': 'facility'
}, inplace=True)
ff_data.rename(str.lower, axis='columns', inplace=True)


In [4]:
ff_sites = ff_data[['facility', 'latitude', 'longitude']]
ff_sites = ff_sites.drop_duplicates(subset=['facility'])
ff_sites.head()

,facility,latitude,longitude
0,169,48.94310,-124.98704
1,227,49.30798,-126.00721
2,520,49.26548,-125.81247
3,1148,49.34182,-125.95328
4,234,49.40478,-126.15072


In [38]:
midpoint = (np.median(ff_sites["latitude"]), np.median(ff_sites["longitude"]))

farm_colours = [[240,249,232],[204,235,197],[168,221,181],[123,204,196],[67,162,202],[8,104,172]]
lightstation_colours = [(254,240,217),(253,212,158),(253,187,132),(252,141,89),(227,74,51),(179,0,0)]
map = pdk.Deck(
    initial_view_state={
        "latitude": midpoint[0],
        "longitude": midpoint[1],
        "zoom": 6,
        "pitch": 50,
    },
    layers=[
        pdk.Layer(
        "HeatmapLayer",
        ff_sites,
        get_position=['longitude', 'latitude'],
        opacity=0.5,
        color_range=farm_colours
        ),
        pdk.Layer(
        "HeatmapLayer",
        ls_active,
        get_position=['longitude', 'latitude'],
        opacity=0.5,
        color_range=lightstation_colours
        ),
    ],
    mapbox_key='pk.eyJ1IjoidG9tc2l0dGVyIiwiYSI6ImNrYjJocjU0azBiNTEycnFwZWFkajB2bnoifQ.M1BDSPntGHfosotYAUAINw'
)

print("Farms are in blue and lightstation locations are in red")
map.to_html('farms.html')

Farms are in blue and lightstation locations are in red


'C:\\Users\\Tom\\Projects\\sea-lice\\farms.html'